In [2]:
import pandas as pd 
import numpy as np 
import sqlite3

In [ ]:
#STRUCTURE
# users table
# accounts table
# account_types table
# transactions table
# transaction_types
# beneficiaries
# loan_applications
# cards
# branches
# employees
# loans
# deposits


In [80]:
def check_table(table_name):
    try:
        q = f"select * from {table_name}"
        print("query created")
    except:
        print('query can not be created')

    try:
        output = conn.execute(q).fetchall()
        print("output received")
    except:
        print(Error)
    print("table data")
    return output

def show_tables (connection):
    try:
        q = """ select * FROM sqlite_master 
                WHERE type='table'"""
    except:
        print(Error)
    return conn.execute(q).fetchall()

def run_query(query, db):
    q = query
    conn = sqlite3.connect(f'{db}.db')
    output = conn.execute(q).fetchall()
    return output

In [42]:
q = """CREATE TABLE IF NOT EXISTS users (
    id INTEGER Primary Key,
    first_name TEXT ,
    last_name TEXT,
    phone_number TEXT,
    pin_code TEXT,
    email TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIME,
    udpated_at TIMESTAMP DEFAULT CURRENT_TIME
)"""


# CREATE TABLE accounts (
#     id INTEGER Primary Key,
#     user_id INTEGER,
#     account_type INTEGER,
#     balance REAL,
#     FOREIGN KEY (user_id) REFERENCES users(id),
#     FOREIGN KEY (account_type) REFERENCES account_types(id)
# )

# CREATE TABLE account_types (
#     id INTEGER Primary Key,
#     account_name TEXT
# )
# """

In [75]:
select_table(table_name= "users")

query created
output received
table data


[]

In [77]:
show_tables(connection=conn)

[('table',
  'users',
  'users',
  2,
  'CREATE TABLE users (\n    id INTEGER Primary Key,\n    first_name TEXT ,\n    last_name TEXT,\n    phone_number TEXT,\n    pin_code TEXT,\n    email TEXT,\n    created_at TIMESTAMP DEFAULT CURRENT_TIME,\n    udpated_at TIMESTAMP DEFAULT CURRENT_TIME\n)')]